# 🎯 Shot Gather 대화형 워크플로우
# Interactive Shot Gather Workflow

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/knocgp/seismic/blob/main/Shot_Gather_Interactive.ipynb)

---

## 📋 단계별 실행 워크플로우

**각 셀을 하나씩 실행하면서 결과를 확인하세요!**

1. ✅ 패키지 설치 및 클래스 정의
2. ✅ 랜덤 합성 모델 생성 → 즉시 시각화
3. ✅ Shot Gather 생성 → 즉시 시각화
4. ✅ 노이즈 추가 → 즉시 시각화
5. ✅ 노이즈 제거 → 즉시 시각화
6. ✅ 전체 비교 → 즉시 시각화
7. ✅ 데이터 저장 및 다운로드
8. ✅ 추가 분석 (트레이스, 스펙트럼)

---

**🚀 사용법: 각 셀을 순서대로 실행 (Shift + Enter)**

## 📦 Step 1: 패키지 설치 및 임포트

In [ ]:
!pip install -q numpy scipy matplotlib

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import median_filter
from typing import Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

print("✅ 패키지 설치 및 임포트 완료!")
print("   - NumPy")
print("   - SciPy")
print("   - Matplotlib")

## 🔧 Step 1-2: ShotGatherProcessor 클래스 정의

In [ ]:
class ShotGatherProcessor:
    """Shot Gather 생성 및 처리 클래스"""
    
    def __init__(self, dt: float = 0.002, nt: int = 1500):
        self.dt = dt
        self.nt = nt
        self.time = np.arange(nt) * dt
        
    def create_random_model(self, nlayers: int = None) -> Dict:
        """완전 랜덤 합성 지반 모델 생성"""
        if nlayers is None:
            nlayers = np.random.randint(4, 9)
        
        model = {'velocity': [], 'density': [], 'thickness': [], 'depth': [], 'name': []}
        
        # 해수층
        water_depth = np.random.uniform(300, 800)
        model['velocity'].append(1500.0)
        model['density'].append(1030.0)
        model['thickness'].append(water_depth)
        model['depth'].append(0.0)
        model['name'].append('Water')
        
        # 해저면
        seabed_vp = np.random.uniform(1600, 2000)
        seabed_rho = np.random.uniform(1900, 2100)
        seabed_thick = np.random.uniform(200, 400)
        model['velocity'].append(seabed_vp)
        model['density'].append(seabed_rho)
        model['thickness'].append(seabed_thick)
        model['depth'].append(water_depth)
        model['name'].append('Seabed')
        
        # 지하 지층들
        current_depth = water_depth + seabed_thick
        for i in range(nlayers - 2):
            if i == 0:
                base_vp = seabed_vp + np.random.uniform(200, 500)
            else:
                base_vp = model['velocity'][-1] + np.random.uniform(100, 600)
            
            vp = base_vp + np.random.normal(0, 100)
            vp = np.clip(vp, 2000, 5000)
            rho = 2000 + (vp - 2000) * 0.2 + np.random.normal(0, 50)
            rho = np.clip(rho, 2000, 2800)
            thickness = np.random.uniform(150, 600)
            
            model['velocity'].append(vp)
            model['density'].append(rho)
            model['thickness'].append(thickness)
            model['depth'].append(current_depth)
            model['name'].append(f'Layer {i+3}')
            current_depth += thickness
        
        return model
    
    def calculate_reflection_coefficients(self, model: Dict):
        velocities = np.array(model['velocity'])
        densities = np.array(model['density'])
        thicknesses = np.array(model['thickness'])
        impedance = velocities * densities
        
        rc = np.zeros(len(velocities) - 1)
        for i in range(len(velocities) - 1):
            rc[i] = (impedance[i+1] - impedance[i]) / (impedance[i+1] + impedance[i])
        
        times = np.zeros(len(velocities) - 1)
        cumulative_time = 0
        for i in range(len(velocities) - 1):
            travel_time = thicknesses[i] / velocities[i]
            cumulative_time += travel_time
            times[i] = cumulative_time * 2
        
        return rc, times
    
    def ricker_wavelet(self, freq: float = 25.0):
        duration = 0.2
        t = np.arange(-duration/2, duration/2, self.dt)
        a = (np.pi * freq * t) ** 2
        wavelet = (1 - 2*a) * np.exp(-a)
        return wavelet / np.max(np.abs(wavelet))
    
    def generate_shot_gather(self, model: Dict, n_traces: int = 48, 
                           offset_min: float = 100, offset_max: float = 2400,
                           freq: float = 25.0):
        """Shot Gather 생성"""
        offsets = np.linspace(offset_min, offset_max, n_traces)
        shot_gather = np.zeros((self.nt, n_traces))
        wavelet = self.ricker_wavelet(freq)
        rc, zero_offset_times = self.calculate_reflection_coefficients(model)
        
        for i_trace, offset in enumerate(offsets):
            reflectivity = np.zeros(self.nt)
            
            for j, (rc_val, t0) in enumerate(zip(rc, zero_offset_times)):
                depths = np.array(model['depth'])
                velocities = np.array(model['velocity'])
                
                if j < len(depths) - 1:
                    avg_depth = depths[j+1]
                    avg_velocity = np.mean(velocities[:j+2])
                    t_nmo = np.sqrt(t0**2 + (offset / avg_velocity)**2)
                    angle = np.arctan(offset / avg_depth)
                    avo_factor = 1 - 0.3 * np.sin(angle)**2
                    
                    idx = int(t_nmo / self.dt)
                    if idx < self.nt:
                        reflectivity[idx] += rc_val * avo_factor
            
            trace = signal.convolve(reflectivity, wavelet, mode='same')
            spreading = 1 / (1 + offset / 1000)
            shot_gather[:, i_trace] = trace * spreading
        
        return shot_gather, offsets
    
    def add_realistic_noise(self, shot_gather, noise_level: float = 0.10):
        """실제적인 노이즈 추가"""
        result = shot_gather.copy()
        signal_power = np.std(shot_gather)
        nt, n_traces = shot_gather.shape
        
        # 백색 잡음
        white_noise = np.random.normal(0, noise_level * signal_power * 0.3, (nt, n_traces))
        result += white_noise
        
        # Ground Roll
        for i in range(5):
            freq = np.random.uniform(5, 15)
            phase_velocity = np.random.uniform(300, 800)
            amplitude = noise_level * signal_power * np.random.uniform(0.5, 1.5)
            
            for j in range(n_traces):
                offset = j * 50
                time_shift = offset / phase_velocity
                phase = 2 * np.pi * freq * (self.time - time_shift)
                ground_roll = amplitude * np.sin(phase + np.random.uniform(0, 2*np.pi))
                decay = np.exp(-self.time / 0.5)
                result[:, j] += ground_roll * decay
        
        # 스파이크 노이즈
        n_spikes = np.random.randint(1, 4)
        for _ in range(n_spikes):
            spike_trace = np.random.randint(0, n_traces)
            spike_time = np.random.randint(0, nt)
            spike_duration = np.random.randint(20, 100)
            if spike_time + spike_duration < nt:
                spike = noise_level * signal_power * 5.0 * np.random.randn(spike_duration)
                result[spike_time:spike_time+spike_duration, spike_trace] += spike
        
        # 저주파 트렌드
        for j in range(n_traces):
            trend_freq = np.random.uniform(0.5, 2.0)
            trend = noise_level * signal_power * 0.4 * np.sin(2 * np.pi * trend_freq * self.time)
            result[:, j] += trend
        
        return result
    
    def denoise_bandpass_filter(self, shot_gather, low_freq: float = 8.0, high_freq: float = 60.0):
        """밴드패스 필터"""
        nt, n_traces = shot_gather.shape
        denoised = np.zeros_like(shot_gather)
        nyquist = 1 / (2 * self.dt)
        low = low_freq / nyquist
        high = high_freq / nyquist
        b, a = signal.butter(4, [low, high], btype='band')
        
        for i in range(n_traces):
            denoised[:, i] = signal.filtfilt(b, a, shot_gather[:, i])
        return denoised
    
    def denoise_fk_filter(self, shot_gather, velocity_cutoff: float = 1500):
        """F-K 필터"""
        nt, n_traces = shot_gather.shape
        fk_spectrum = np.fft.fft2(shot_gather)
        fk_spectrum_shifted = np.fft.fftshift(fk_spectrum)
        
        freq = np.fft.fftshift(np.fft.fftfreq(nt, self.dt))
        k = np.fft.fftshift(np.fft.fftfreq(n_traces, 50))
        
        fk_filter = np.ones_like(fk_spectrum_shifted)
        for i, f in enumerate(freq):
            for j, kval in enumerate(k):
                if f != 0 and kval != 0:
                    apparent_velocity = abs(f / kval)
                    if apparent_velocity < velocity_cutoff:
                        fk_filter[i, j] = 0.1
        
        fk_filtered = fk_spectrum_shifted * fk_filter
        fk_filtered_unshifted = np.fft.ifftshift(fk_filtered)
        return np.real(np.fft.ifft2(fk_filtered_unshifted))
    
    def denoise_median_filter(self, shot_gather, size: int = 5):
        """Median 필터"""
        return median_filter(shot_gather, size=(size, 1))
    
    def denoise_combined(self, shot_gather):
        """조합 노이즈 제거"""
        result = self.denoise_bandpass_filter(shot_gather, 8.0, 60.0)
        result = self.denoise_fk_filter(result, 1500)
        result = self.denoise_median_filter(result, 5)
        return result
    
    def plot_model(self, model: Dict):
        """지층 모델 시각화"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 8))
        
        depths = model['depth']
        velocities = model['velocity']
        densities = model['density']
        
        for i in range(len(depths)):
            depth_top = depths[i]
            depth_bottom = depths[i] + model['thickness'][i]
            
            ax1.fill_between([velocities[i]-100, velocities[i]+100],
                            depth_top, depth_bottom,
                            alpha=0.4, label=model['name'][i] if i < 5 else None)
            ax1.plot([velocities[i], velocities[i]], [depth_top, depth_bottom],
                    'b-', linewidth=2.5)
            
            ax2.fill_between([densities[i]-50, densities[i]+50],
                            depth_top, depth_bottom,
                            alpha=0.4)
            ax2.plot([densities[i], densities[i]], [depth_top, depth_bottom],
                    'r-', linewidth=2.5)
        
        ax1.set_xlabel('Velocity (m/s)', fontsize=13, fontweight='bold')
        ax1.set_ylabel('Depth (m)', fontsize=13, fontweight='bold')
        ax1.set_title('Velocity Model', fontsize=15, fontweight='bold')
        ax1.invert_yaxis()
        ax1.grid(True, alpha=0.4)
        ax1.legend(fontsize=10)
        
        ax2.set_xlabel('Density (kg/m³)', fontsize=13, fontweight='bold')
        ax2.set_ylabel('Depth (m)', fontsize=13, fontweight='bold')
        ax2.set_title('Density Model', fontsize=15, fontweight='bold')
        ax2.invert_yaxis()
        ax2.grid(True, alpha=0.4)
        
        plt.tight_layout()
        plt.show()
    
    def plot_shot_gather(self, shot_gather, offsets, title: str = "Shot Gather", clip_percentile: float = 99):
        """Shot Gather 시각화"""
        fig, ax = plt.subplots(figsize=(14, 10))
        vmax = np.percentile(np.abs(shot_gather), clip_percentile)
        
        for i, offset in enumerate(offsets):
            trace = shot_gather[:, i]
            trace_scaled = trace / vmax * 30
            ax.plot(offset + trace_scaled, self.time, 'k-', linewidth=0.3)
            ax.fill_betweenx(self.time, offset, offset + trace_scaled,
                            where=(trace_scaled > 0), color='black', alpha=0.6)
        
        ax.set_xlabel('Offset (m)', fontsize=13, fontweight='bold')
        ax.set_ylabel('Time (s)', fontsize=13, fontweight='bold')
        ax.set_title(title, fontsize=15, fontweight='bold')
        ax.invert_yaxis()
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_xlim([offsets[0] - 100, offsets[-1] + 100])
        plt.tight_layout()
        plt.show()
    
    def plot_comparison(self, original, noisy, denoised, offsets):
        """3개 비교"""
        fig, axes = plt.subplots(1, 3, figsize=(20, 10))
        titles = ['Original (Clean)', 'With Noise', 'Denoised']
        data_list = [original, noisy, denoised]
        vmax = np.percentile(np.abs(original), 99)
        
        for ax, data, title in zip(axes, data_list, titles):
            for i, offset in enumerate(offsets):
                trace = data[:, i]
                trace_scaled = trace / vmax * 30
                ax.plot(offset + trace_scaled, self.time, 'k-', linewidth=0.3)
                ax.fill_betweenx(self.time, offset, offset + trace_scaled,
                                where=(trace_scaled > 0), color='black', alpha=0.6)
            
            ax.set_xlabel('Offset (m)', fontsize=12, fontweight='bold')
            ax.set_ylabel('Time (s)', fontsize=12, fontweight='bold')
            ax.set_title(title, fontsize=14, fontweight='bold')
            ax.invert_yaxis()
            ax.grid(True, alpha=0.3, linestyle='--')
            ax.set_xlim([offsets[0] - 100, offsets[-1] + 100])
        
        plt.tight_layout()
        plt.show()

print("✅ ShotGatherProcessor 클래스 정의 완료!")

## 🌍 Step 2: 랜덤 합성 모델 생성

**이 셀을 실행하면:**
- 완전 랜덤 지층 모델 생성
- 지층 정보 테이블 출력
- 속도 및 밀도 프로파일 시각화

In [ ]:
# 프로세서 초기화
processor = ShotGatherProcessor(dt=0.002, nt=1500)
print("✅ 프로세서 초기화 완료")
print(f"   - 샘플링 간격: {processor.dt*1000:.1f} ms")
print(f"   - 시간 샘플: {processor.nt}개")
print(f"   - 총 시간: {processor.time[-1]:.2f} s")
print()

# 랜덤 모델 생성
print("🌍 랜덤 합성 지반 모델 생성 중...")
model = processor.create_random_model(nlayers=6)
print("✅ 모델 생성 완료!")
print()

# 지층 정보 출력
print("="*80)
print("📊 생성된 지층 정보")
print("="*80)
print(f"{'Layer':<15} {'Depth (m)':<12} {'Thickness (m)':<15} {'Velocity (m/s)':<15} {'Density (kg/m³)'}")
print("-"*80)
for i in range(len(model['name'])):
    print(f"{model['name'][i]:<15} {model['depth'][i]:<12.1f} {model['thickness'][i]:<15.1f} "
          f"{model['velocity'][i]:<15.1f} {model['density'][i]:<15.1f}")
print("="*80)
print()

# 모델 시각화
print("📈 지층 모델 시각화...")
processor.plot_model(model)

## 🎯 Step 3: Shot Gather 생성 (Clean)

**이 셀을 실행하면:**
- 48개 트레이스 Shot Gather 생성
- NMO 및 AVO 효과 적용
- 원본 Shot Gather 시각화

In [ ]:
print("🎯 Shot Gather 생성 중...")
print()

# Shot Gather 생성
n_traces = 48
offset_min = 100
offset_max = 2400
freq = 25.0

clean_shot, offsets = processor.generate_shot_gather(
    model,
    n_traces=n_traces,
    offset_min=offset_min,
    offset_max=offset_max,
    freq=freq
)

print("✅ Shot Gather 생성 완료!")
print()
print("📊 Shot Gather 정보:")
print(f"   - 트레이스 개수: {n_traces}")
print(f"   - 오프셋 범위: {offsets[0]:.0f} ~ {offsets[-1]:.0f} m")
print(f"   - Wavelet 주파수: {freq} Hz")
print(f"   - 데이터 크기: {clean_shot.shape}")
print(f"   - RMS: {np.sqrt(np.mean(clean_shot**2)):.6f}")
print()

# 시각화
print("📈 원본 Shot Gather 시각화...")
processor.plot_shot_gather(clean_shot, offsets, "✨ Original Shot Gather (Clean)")

## 📢 Step 4: 노이즈 추가

**이 셀을 실행하면:**
- 4가지 노이즈 추가 (백색잡음, Ground Roll, 스파이크, 저주파)
- 노이즈가 추가된 Shot Gather 시각화
- SNR 계산

In [ ]:
print("📢 노이즈 추가 중...")
print()

# 노이즈 추가
noise_level = 0.12
noisy_shot = processor.add_realistic_noise(clean_shot, noise_level=noise_level)

print("✅ 노이즈 추가 완료!")
print()
print("📊 추가된 노이즈:")
print("   ✅ 백색 잡음 (White Noise)")
print("   ✅ Ground Roll (5-15 Hz)")
print("   ✅ 스파이크 노이즈 (Bad Traces)")
print("   ✅ 저주파 트렌드")
print()

# 통계
noise = noisy_shot - clean_shot
snr_before = 20 * np.log10(np.std(clean_shot) / np.std(noise))

print("📈 통계:")
print(f"   - Clean RMS: {np.sqrt(np.mean(clean_shot**2)):.6f}")
print(f"   - Noisy RMS: {np.sqrt(np.mean(noisy_shot**2)):.6f}")
print(f"   - Noise RMS: {np.sqrt(np.mean(noise**2)):.6f}")
print(f"   - SNR: {snr_before:.2f} dB")
print()

# 시각화
print("📈 노이즈가 추가된 Shot Gather 시각화...")
processor.plot_shot_gather(noisy_shot, offsets, "📢 Shot Gather with Noise")

## 🔧 Step 5: 노이즈 제거

**이 셀을 실행하면:**
- 밴드패스 필터 (8-60 Hz)
- F-K 필터 (Ground Roll 제거)
- Median 필터 (스파이크 제거)
- 노이즈 제거된 Shot Gather 시각화
- SNR 개선 계산

In [ ]:
print("🔧 노이즈 제거 중...")
print()
print("적용 기법:")
print("   1️⃣ 밴드패스 필터 (8-60 Hz)")
print("   2️⃣ F-K 필터 (Ground Roll 제거, < 1500 m/s)")
print("   3️⃣ Median 필터 (스파이크 제거)")
print()

# 노이즈 제거
denoised_shot = processor.denoise_combined(noisy_shot)

print("✅ 노이즈 제거 완료!")
print()

# 통계
residual = denoised_shot - clean_shot
snr_after = 20 * np.log10(np.std(clean_shot) / np.std(residual))

print("="*80)
print("📊 노이즈 제거 결과")
print("="*80)
print(f"Denoised RMS:    {np.sqrt(np.mean(denoised_shot**2)):.6f}")
print(f"Residual RMS:    {np.sqrt(np.mean(residual**2)):.6f}")
print()
print(f"SNR (노이즈 추가 후):  {snr_before:.2f} dB")
print(f"SNR (노이즈 제거 후):  {snr_after:.2f} dB")
print(f"SNR 개선:             {snr_after - snr_before:.2f} dB  ⬆️")
print("="*80)
print()

# 시각화
print("📈 노이즈 제거된 Shot Gather 시각화...")
processor.plot_shot_gather(denoised_shot, offsets, "🔧 Denoised Shot Gather")

## 📊 Step 6: 전체 비교

**이 셀을 실행하면:**
- 원본, 노이즈, 노이즈 제거 3개 나란히 비교
- 시각적으로 차이 확인

In [ ]:
print("📊 전체 비교 시각화...")
print()

processor.plot_comparison(clean_shot, noisy_shot, denoised_shot, offsets)

print("✅ 비교 완료!")
print()
print("💡 주의 깊게 보세요:")
print("   - Original: 깨끗한 반사 신호")
print("   - With Noise: Ground Roll과 스파이크가 보입니다")
print("   - Denoised: 노이즈가 제거되고 원본에 가까워졌습니다")

## 💾 Step 7: 데이터 저장 및 다운로드

**이 셀을 실행하면:**
- 3개 NPZ 파일 저장
- 자동 다운로드 (Colab)

In [ ]:
print("💾 데이터 저장 중...")
print()

# 데이터 저장
np.savez('shot_gather_clean.npz',
         shot_gather=clean_shot,
         offsets=offsets,
         time=processor.time,
         model=model)
print("✅ shot_gather_clean.npz 저장 완료")

np.savez('shot_gather_noisy.npz',
         shot_gather=noisy_shot,
         offsets=offsets,
         time=processor.time,
         model=model)
print("✅ shot_gather_noisy.npz 저장 완료")

np.savez('shot_gather_denoised.npz',
         shot_gather=denoised_shot,
         offsets=offsets,
         time=processor.time,
         model=model)
print("✅ shot_gather_denoised.npz 저장 완료")
print()

# Colab에서 다운로드
try:
    from google.colab import files
    print("📥 다운로드 시작...")
    print()
    files.download('shot_gather_clean.npz')
    files.download('shot_gather_noisy.npz')
    files.download('shot_gather_denoised.npz')
    print("✅ 모든 파일 다운로드 완료!")
except:
    print("ℹ️ 로컬 환경에서 실행 중 - 파일이 현재 디렉토리에 저장되었습니다.")

print()
print("="*80)
print("🎉 전체 워크플로우 완료!")
print("="*80)

## 🔬 Step 8: 추가 분석 - 단일 트레이스 비교

**선택사항: 특정 트레이스를 자세히 분석**

In [ ]:
# 중간 오프셋 트레이스 선택
trace_idx = len(offsets) // 2

print(f"🔬 트레이스 {trace_idx} 분석 (오프셋 = {offsets[trace_idx]:.0f} m)")
print()

# 플롯
fig, axes = plt.subplots(1, 3, figsize=(18, 8))

axes[0].plot(clean_shot[:, trace_idx], processor.time, 'b-', linewidth=1.5)
axes[0].set_xlabel('Amplitude', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Time (s)', fontsize=12, fontweight='bold')
axes[0].set_title(f'Clean Trace\n(Offset={offsets[trace_idx]:.0f}m)', fontsize=13, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3)

axes[1].plot(noisy_shot[:, trace_idx], processor.time, 'r-', linewidth=1.5)
axes[1].set_xlabel('Amplitude', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Time (s)', fontsize=12, fontweight='bold')
axes[1].set_title(f'Noisy Trace\n(Offset={offsets[trace_idx]:.0f}m)', fontsize=13, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3)

axes[2].plot(denoised_shot[:, trace_idx], processor.time, 'g-', linewidth=1.5)
axes[2].set_xlabel('Amplitude', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Time (s)', fontsize=12, fontweight='bold')
axes[2].set_title(f'Denoised Trace\n(Offset={offsets[trace_idx]:.0f}m)', fontsize=13, fontweight='bold')
axes[2].invert_yaxis()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"✅ 트레이스 {trace_idx} 비교 완료!")

## 📊 Step 9: 주파수 스펙트럼 분석

**선택사항: 주파수 도메인에서 비교**

In [ ]:
# 중간 트레이스 FFT
trace_idx = len(offsets) // 2
dt = processor.time[1] - processor.time[0]

print(f"📊 주파수 스펙트럼 분석 (트레이스 {trace_idx}, 오프셋 {offsets[trace_idx]:.0f}m)")
print()

# FFT 계산
freq = np.fft.rfftfreq(len(processor.time), dt)
clean_fft = np.abs(np.fft.rfft(clean_shot[:, trace_idx]))
noisy_fft = np.abs(np.fft.rfft(noisy_shot[:, trace_idx]))
denoised_fft = np.abs(np.fft.rfft(denoised_shot[:, trace_idx]))

# 플롯
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(freq, clean_fft, 'b-', linewidth=2, label='Clean', alpha=0.8)
ax.plot(freq, noisy_fft, 'r-', linewidth=2, label='Noisy', alpha=0.6)
ax.plot(freq, denoised_fft, 'g-', linewidth=2, label='Denoised', alpha=0.8)

ax.set_xlabel('Frequency (Hz)', fontsize=13, fontweight='bold')
ax.set_ylabel('Amplitude Spectrum', fontsize=13, fontweight='bold')
ax.set_title(f'Frequency Spectrum Comparison (Offset={offsets[trace_idx]:.0f}m)', 
             fontsize=14, fontweight='bold')
ax.set_xlim([0, 100])
ax.legend(fontsize=12, loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ 주파수 스펙트럼 분석 완료!")
print()
print("💡 관찰 포인트:")
print("   - Noisy (빨강): 저주파 및 고주파에 노이즈가 많습니다")
print("   - Denoised (초록): 필터링으로 노이즈가 제거되었습니다")
print("   - Clean (파랑): 원본 신호의 주파수 특성")

## 🎉 완료!

---

### 📝 요약

모든 단계를 실행하셨다면:

✅ 랜덤 지층 모델 생성 완료  
✅ Shot Gather 생성 완료  
✅ 노이즈 추가 완료  
✅ 노이즈 제거 완료  
✅ 3개 파일 다운로드 완료

---

### 💾 생성된 파일

1. **shot_gather_clean.npz** - 원본 Shot Gather
2. **shot_gather_noisy.npz** - 노이즈 추가된 Shot Gather
3. **shot_gather_denoised.npz** - 노이즈 제거된 Shot Gather

---

### 🔄 다시 실행하기

새로운 랜덤 모델로 다시 실행하려면:
- **Step 2**부터 다시 실행하세요!
- 매번 다른 지층 모델과 결과를 얻을 수 있습니다.

---

### 📚 관련 링크

- **GitHub**: https://github.com/knocgp/seismic
- **상세 가이드**: [SHOT_GATHER_GUIDE.md](https://github.com/knocgp/seismic/blob/main/SHOT_GATHER_GUIDE.md)
- **Colab 가이드**: [COLAB_GUIDE_KR.md](https://github.com/knocgp/seismic/blob/main/COLAB_GUIDE_KR.md)

---

**Made with ❤️ for Seismic Data Processing**